In [ ]:
import hypermri
import hypermri.utils.utils_anatomical as ut_anat
import sys

import hypermri.utils.utils_spectroscopy as ut_spec
import hypermri.utils.utils_fitting as ut_fitting
import hypermri.utils.utils_general as utg
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import datetime
from matplotlib.patches import Rectangle
from scipy.optimize import curve_fit 
from hypermri.utils.utils_sv_spectroscopy import Plot_Voxel_on_Anat
from scipy.stats import rayleigh
# Autoreload extension so that you dont have to reload the kernel every time something is changed in the hypermri or magritek folders
%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib
from matplotlib import rc
rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
rc("text", usetex=True)
matplotlib.rcParams.update({"font.size": 11})

from Template import import_paths

basepath,savepath,publication_path=import_paths()

# Load Data

In [ ]:
# define scan path
dirpath = basepath + '/'
scans = hypermri.BrukerDir(dirpath,verbose=True)

animal_ID=''
weight=
date=''
age=

locations=['kidney l up','kidney r down','kidney l down','kidney r up','muscle back','artery','liver','muscle foot','kidney l up redone','kidney r redone']

In [ ]:
slaser_diss1 = scans[11]
sp2_diss1=scans[16]
sp90_diss1=scans[17]
axial=scans[25]
coronal=scans[18]
mvpress_shimcheck=scans[22]
press_diss2 = scans[20]
sp2_diss2=scans[19]
sp90_diss2=scans[21]

# Meta data

In [ ]:
TR=slaser_diss1.method['PVM_RepetitionTime']/1000
NR=slaser_diss1.method['PVM_NRepetitions']
FA=slaser_diss1.method['VoxPul1'][2]
voxsize=slaser_diss1.method['PVM_VoxArrSize']
nvox=slaser_diss1.method['PVM_NVoxels']

# Anatomicals

In [ ]:
plt.close('all')
fig,ax=plt.subplots(1,7,figsize=(12,4))

for n in range(nvox-3):
    Plot_Voxel_on_Anat(slaser_diss1,coronal,ax[n],n,vox_color='C'+str(n),plot_number=False,vmin=0,vmax=55)

    ax[n].axis('off')
    ax[n].set_ylim([-15,20])
    ax[n].set_xlim([13,-11])
    
    ax[n].set_title(None)
    fig.subplots_adjust(wspace=0.05)
    Rect=Rectangle((5,-13),5,1,fc='w')
    ax[0].text(10,-11,'5mm',c='w')
    ax[n].add_patch(Rect)


In [ ]:
fig,ax=plt.subplots(1,7,figsize=(12,3))

for n in range(nvox-3):
    Plot_Voxel_on_Anat(slaser_diss1,axial,ax[n],n,vox_color='C'+str(n),plot_number=False,vmin=0,vmax=40)

    ax[n].axis('off')
    ax[n].set_ylim([-15,10])
    ax[n].set_xlim([13,-11])
    
    ax[n].set_title(None)
    fig.subplots_adjust(wspace=0.05)
    Rect=Rectangle((7,-11),5,1,fc='w')
    ax[0].text(12,-9,'5mm',c='w')
    ax[n].add_patch(Rect)


# Plot shimcheck

In [ ]:
fig,ax=plt.subplots(4,2,tight_layout=True,figsize=(4,5))
proton_shim_mvpress=np.abs(np.squeeze(mvpress_shimcheck.get_fids_spectra(0,70)[0]))
proton_shim_ppm=mvpress_shimcheck.get_ppm(70)

for n in range(8):
    nx,ny=n//2,n%2
    ax[nx,ny].plot(proton_shim_ppm,(proton_shim_mvpress[:,n]-np.mean(proton_shim_mvpress[50:100,n]))/np.std(proton_shim_mvpress[50:100,n]))
    #ax.plot(proton_shim_ppm[50:100],(proton_shim_mvpress[50:100,n]-np.mean(proton_shim_mvpress[50:100,n]))/np.std(proton_shim_mvpress[50:100,n]),color='r')
    ax[nx,ny].set_xlim([9,1])
    ax[nx,ny].set_ylim([-50,750])
    ax[nx,0].set_ylabel('I [a.u.]')
    ax[nx,1].set_yticks([])
    
    ax[nx,ny].set_xticks([])
    ax[nx,ny].set_title(locations[n])
ax[3,0].set_xticks([8,6,4,2])
ax[3,1].set_xticks([8,6,4,2])
ax[3,0].set_xlabel(r'$\sigma$ [ppm]')
ax[3,1].set_xlabel(r'$\sigma$ [ppm]')


# Spectra plots

In [ ]:
fig,(ax,ax2)=plt.subplots(2,1,figsize=(3,4),tight_layout=True)
sp2_diss1_spec=sp2_diss1.get_spec(5,70)[2]
sp2_diss2_spec=sp2_diss2.get_spec(5,70)[2]


sp90_diss1_spec=sp90_diss1.get_spec(5,70)[2]
sp90_diss2_spec=sp90_diss2.get_spec(5,70)[2]
max_val=np.max(np.abs(sp2_diss1_spec))
ax.plot(sp2_diss1.get_ppm(70,0),np.abs(np.squeeze(sp2_diss1_spec))/max_val,label='sLASER')
ax.plot(sp2_diss2.get_ppm(70,0),np.abs(np.squeeze(sp2_diss2_spec))/max_val,label='PRESS')
ax.set_xlabel('ppm')
ax.set_xlim([200,155])
ax.set_yticks([0,0.5,1])
ax.legend()

ax.set_title('2° SP before MRS')

max_val=np.max(np.abs(sp90_diss1_spec))
ax2.plot(sp90_diss1.get_ppm(70,0),np.abs(np.squeeze(sp90_diss1_spec))/max_val,label='sLASER')
ax2.plot(sp90_diss2.get_ppm(70,0),np.abs(np.squeeze(sp90_diss2_spec))/max_val,label='PRESS')
ax2.set_xlabel('ppm')
ax2.set_xlim([200,155])
ax2.set_yticks([0,0.5,1])
ax2.legend()
ax2.set_title('90° SP after MRS')


In [ ]:
plt.close('all')
fig,ax=plt.subplots(1,2,figsize=(11.5/2.54,7.0/2.54),tight_layout=True)
locations=['Kidney L1','Kidney R1','Kidney L2','Kidney R2','Back muscle','Bloodvessel','Liver','muscle foot','kidney l up redone','kidney r redone'][:7]

slaser_spec=np.abs(np.squeeze(slaser_diss1.get_fids_spectra(5,70)[0]))
ppm=slaser_diss1.get_ppm(70)
press_spec=np.abs(np.squeeze(press_diss2.get_fids_spectra(5,70)[0]))

ppm_press=press_diss2.get_ppm(70)

[ax[0].plot(ppm,((slaser_spec[:,n]-np.mean(slaser_spec[0:50,n]))/np.std(slaser_spec[0:50,n]))+50*n,label=locations[n],color='C'+str(n)) for n in range(len(locations))]
[ax[1].plot(ppm,((press_spec[:,n]-np.mean(press_spec[0:50,n]))/np.std(press_spec[0:50,n]))+50*n,label=locations[n],color='C'+str(n)) for n in range(len(locations))]

for n in range(2):
    ax[n].set_xlim([190,160])
    ax[n].set_xlabel(r'$\sigma$[ppm]')
    ax[n].set_yticks([])
    ax[n].set_xticks([165,170,175,180,185])

# Compute SNR and fit for both datasets

### Fit both datasets

In [ ]:
def fit_data(experiment):
    metabs = ['pyruvate', 'lactate','pyruvatehydrate','alanine']
    fit_params = {}
    fit_params["zoomfactor"] = 1.5
    fit_params["max_t2_s"] = 0.5
    fit_params["min_t2_s"] = 0.001
    fit_params["range_t2s_s"] = 0.05

    # get the indices of the peaks:
    fit_params["metabs"] = metabs
    fit_params["fit_range_repetitions"] = 1
    fit_params["range_freqs_Hz"] = 25
    fit_params["cut_off"] = 70
    fit_params["niter"] = 1 # number of iterations:
    fit_params["npoints"] = 21 # number of tested points per iteration:
    fit_params["rep_fitting"] = 11 # number of tested points per iteration:
    fit_params["provided_dims"] = ["fid","repetitions"]


    fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)

    cut_off_spec=np.fft.fftshift(np.fft.fft(experiment.complex_fids[fit_params["cut_off"]:,:],axis=0),axes=(0,))

    fit_params = ut_fitting.def_fit_params(fit_params=fit_params, data_obj=experiment)

    fit_spectrums, fit_amps, fit_freqs, fit_t2s, fit_stds  = ut_fitting.fit_data_pseudo_inv(input_data=cut_off_spec,
                                                                                 data_obj=experiment,
                                                                      fit_params=fit_params,
                                                                  use_multiprocessing=True)

    fit_freqs_ppm = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_freqs), hz_axis=fit_params["freq_range_Hz"], ppm_axis=fit_params["freq_range_ppm"], ppm_axis_flipped=False)
    fit_stds_ppm = ut_spec.freq_Hz_to_ppm(freq_Hz=np.squeeze(fit_stds), hz_axis=fit_params["freq_range_Hz"], ppm_axis=fit_params["freq_range_ppm"], ppm_axis_flipped=False)

    fig,ax=plt.subplots(len(locations),1,tight_layout=True,figsize=(6,8))

    [ax[n].plot(fit_params['freq_range_ppm'],np.abs(np.sum(np.squeeze(fit_spectrums)[:,n,:],axis=1)),color='r') for n in range(len(locations))]
    [ax[n].plot(fit_params['freq_range_ppm'],np.abs(np.squeeze(cut_off_spec)[:,n]),color='k',alpha=0.5) for n in range(len(locations))]
    [ax[n].set_xlim([185,165]) for n in range(len(locations))]
    [ax[n].set_xticks([185,180,175,170,165]) for n in range(len(locations))]
    [ax[n].set_title(locations[n]) for n in range(len(locations))]

    pyr_amp=np.abs(np.squeeze(fit_amps[:,:,:,:,:,:,0]))
    lac_amp=np.abs(np.squeeze(fit_amps[:,:,:,:,:,:,1]))
    d_pyr_amp = np.squeeze(fit_stds[..., 0, 0])
    d_lac_amp = np.squeeze(fit_stds[..., 1, 0])
    return fit_spectrums, fit_amps, fit_freqs, fit_t2s, fit_stds,fit_freqs_ppm,fit_stds_ppm,cut_off_spec,fit_params,metabs



In [ ]:
fit_spectrums_slaser, fit_amps_slaser, fit_freqs_slaser, fit_t2s_slaser, fit_stds_slaser,fit_freqs_ppm_slaser,fit_stds_ppm_slaser,cut_off_spec_slaser,fit_params_slaser,metabs_slaser=fit_data(slaser_diss1)

In [ ]:
fit_spectrums_press, fit_amps_press, fit_freqs_press, fit_t2s_press, fit_stds_press,fit_freqs_ppm_press,fit_stds_ppm_press,cut_off_spec_press,fit_params_press,metabs_press=fit_data(press_diss2)

### Compute SNR

In [ ]:
def compute_snr(experiment,cutoff_spec,fit_spectrums,nvox=7):
    metabs = ['pyruvate', 'lactate','pyruvatehydrate','alanine']
    fig,ax=plt.subplots(nvox,2,tight_layout=True,figsize=(6,10))
    noise_floor=np.ones((nvox))
    for n in range(nvox):
        ax[n,0].plot(np.abs((np.squeeze(cutoff_spec)[:,n]-np.mean(np.abs(np.squeeze(cutoff_spec)[0:150,n])))/np.std(np.abs(np.squeeze(cutoff_spec)[0:150,n]))))
        ax[n,0].plot(np.abs((np.squeeze(cutoff_spec)[:,n]-np.mean(np.abs(np.squeeze(cutoff_spec)[0:150,n])))/np.std(np.abs(np.squeeze(cutoff_spec)[0:150,n])))[0:150])
        noise_spec=np.abs(((np.squeeze(cutoff_spec)[:,n]-np.mean(np.real(np.squeeze(cutoff_spec)[0:150,n])))/np.std(np.real(np.squeeze(cutoff_spec)[0:150,n]))))[0:150]

        params = rayleigh.fit(noise_spec)
        scale = params[1]

        # Define x values for plotting the fitted distribution
        x = np.linspace(0, max(noise_spec), 100)
        pdf_fitted = rayleigh.pdf(x, loc=0, scale=scale)


        ax[n,1].hist(noise_spec, bins=30, density=True, alpha=0.3, color='C0', edgecolor='black')
        ax[n,1].plot(x, pdf_fitted, 'r-', label=f'Rayleigh fit (scale={scale:.2f})')
        # Create a frozen Rayleigh distribution object with the fitted scale parameter
        fitted_rayleigh = rayleigh(scale=scale)

        # Get mean and standard deviation
        mean = fitted_rayleigh.mean()
        std_dev = fitted_rayleigh.std()
        noise_floor[n]=mean+std_dev
        ax[n,1].set_title(str(mean.round(1))+'±'+str(std_dev.round(1)))
    
    
    fig,ax=plt.subplots(nvox,2,tight_layout=True,figsize=(7,5*nvox/2))
    peak_snrs=np.ones((4,nvox))*np.nan
    mean_noise=[np.mean(np.abs(np.squeeze(cutoff_spec)[0:150,n])) for n in range(nvox)]
    std_noise=[np.std(np.abs(np.squeeze(cutoff_spec)[0:150,n])) for n in range(nvox)]

    for voxel in range(nvox):
        ax[voxel,0].plot(experiment.get_ppm(70),np.abs(np.squeeze(cutoff_spec))[:,voxel])
        ax[voxel,0].plot(experiment.get_ppm(70),np.sum(np.abs(np.squeeze(fit_spectrums)[:,voxel,:]),axis=1),color='r')
        ax[voxel,1].plot(experiment.get_ppm(70),np.abs(np.squeeze(cutoff_spec))[:,voxel],color='k',alpha=0.3)

        for peak in range(4):
            max_peak_fit_val=np.max(np.abs(np.squeeze(fit_spectrums)[:,voxel,peak]))

            snr=np.round((max_peak_fit_val-mean_noise[voxel])/std_noise[voxel],2)
            peak_snrs[peak,voxel]=snr
            ax[voxel,1].plot(experiment.get_ppm(70),np.abs(np.squeeze(fit_spectrums)[:,voxel,peak]),label=metabs[peak]+',SNR='+str(snr))
            ax[voxel,1].set_title('Noise='+str(np.round(noise_floor[voxel],0)))
        ax[voxel,0].set_xlim([195,155])
        ax[voxel,1].set_xlim([195,155])

        ax[voxel,1].legend()
        ax[voxel,0].set_title(locations[voxel])
    
    return peak_snrs,noise_floor


In [ ]:
peak_snrs_slaser,noise_slaser=compute_snr(slaser_diss1,cut_off_spec_slaser,fit_spectrums_slaser)

In [ ]:
peak_snrs_press,noise_press=compute_snr(press_diss2,cut_off_spec_press,fit_spectrums_press)

In [ ]:
output_df_slaser=pd.DataFrame(columns=['Voxel','SNR Pyr','SNR Lac','SNR Hydr','SNR Ala','Noise'])
output_df_slaser['Voxel']=locations
output_df_slaser['SNR Pyr']=peak_snrs_slaser[0,:]
output_df_slaser['SNR Lac']=peak_snrs_slaser[1,:]
output_df_slaser['SNR Hydr']=peak_snrs_slaser[2,:]
output_df_slaser['SNR Ala']=peak_snrs_slaser[3,:]
output_df_slaser['Noise']=noise_slaser
output_df_slaser.round(0)

In [ ]:
output_df_press=pd.DataFrame(columns=['Voxel','SNR Pyr','SNR Lac','SNR Hydr','SNR Ala','Noise'])
output_df_press['Voxel']=locations
output_df_press['SNR Pyr']=peak_snrs_press[0,:]
output_df_press['SNR Lac']=peak_snrs_press[1,:]
output_df_press['SNR Hydr']=peak_snrs_press[2,:]
output_df_press['SNR Ala']=peak_snrs_press[3,:]
output_df_press['Noise']=noise_press
output_df_press.round(0)

# FWHM values

In [ ]:
fwhms_slaser=1/fit_t2s_slaser.squeeze()
fwhms_press=1/fit_t2s_press.squeeze()

fwhm_df=pd.DataFrame(columns=['Voxel','Sequence','Pyr','Lac','Hydr','Ala'])
fwhm_df['Voxel']=locations*2
seq=[]
for n in range(len(locations)*2):
    if n<len(locations):
        seq.append('sLASER')
    else:
        seq.append('PRESS')
fwhm_df['Sequence']=seq


fwhm_df['Pyr']=np.concatenate([np.array(fwhms_slaser[:7,0]),np.array(fwhms_press[:7,0])])
fwhm_df['Lac']=np.concatenate([np.array(fwhms_slaser[:7,1]),np.array(fwhms_press[:7,1])])
fwhm_df['Hydr']=np.concatenate([np.array(fwhms_slaser[:7,2]),np.array(fwhms_press[:7,2])])
fwhm_df['Ala']=np.concatenate([np.array(fwhms_slaser[:7,3]),np.array(fwhms_press[:7,3])])
fwhm_df.round(1)